## MinMax Algorithm

## From definitions to know-how

In [1]:
from collections import defaultdict
import random
import math
import functools 

### Task 0


In [2]:
class Board(defaultdict):
    empty = '.'
    used = '#'
    
    def __init__(self, width=8, height=8, current_player=None, **kwds):
        self.__dict__.update(width=width, height=height, current_player=current_player, **kwds)
 
    def __missing__(self, pos):
        (x,y) = pos
        if (x>=0) and (y>=0) and (x<self.width) and (y<self.height):
            return self.empty
        else:
            return self.used       
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.current_player)
    
    def __repr__(self):
        def row(y): 
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'
    
    def update_board(self, changes: dict, **kwds) -> 'Board':
        board = Board(width = self.width, height = self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

board = Board()
print(board.__repr__)
print(board.update_board({(2,3): 'X'}))
print(board.update_board({(6,3): 'O'}))

<bound method Board.__repr__ of . . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
>
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . X . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .

. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . O .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .



### Task 1


In [3]:
def k_in_row(board, player, square, k):
    
    def in_row(x,y,xpos,ypos):
        if board[x,y] != player:
            return 0
        else:
            return 1 + in_row(x + xpos, y + ypos,xpos,ypos)
    
    
    lista = [] 
    for xpos, ypos in (1, 1),(0, 1),(1, 0),(1, -1):  
        lista.append(in_row(*square, xpos, ypos) + in_row(*square, -xpos, -ypos) - 1 >= k) 
    for i in lista:
        if i:
            return True
    return False 

### Task 2


In [5]:
class TicTacToe:
    
    def __init__(self, height=3, width=3, k=3):
        self.k = k
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, current_player='X', utility=0)
        # The board where X plays first, and the utility value is 0.
 
    def actions(self, board):
        return self.squares - set(board)
    

    def utility(self, board, player):
        if player == 'X':
            return board.utility 
        elif player == 'O':
            return -board.utility
        else:
            return 0

    def make_move(self, board, square):
        player = board.current_player
        if player == 'O':
            board = board.update_board({square: player}, current_player='X')
        else:
            board = board.update_board({square: player}, current_player='O')
        win = k_in_row(board, player, square, self.k)
        if win:
            if player == 'X':
                board.utility = 1
            else:
                board.utility = -1
            
        else:
            board.utility = 0
        
        return board
    
    def end(self, board):
        if (board.utility != 0) or (len(self.squares)==len(board)):
            return True
        else:
            return False
 
    def draw_board(self, board):
        print(board)

In [6]:
def random_player(game, state):
    return random.choice(list(game.actions(state)))

In [7]:
def player(search_algorithm):
    return lambda game, state: search_algorithm(game, state)[1]

### Task 3

In [13]:
def play_game(game, strategies: dict):
    state = game.initial
    while game.end(state)==False:
        player = state.current_player
        move = strategies[player](game, state)
        state = game.make_move(state, move)
        print('Player:',player)
        print('Move:',move)
        print(state)
            
    return state

## Min-Max Algorithm

In [14]:

infinity = math.inf

def minimax_search(game, state):
    player = state.current_player

    def max_value(state):
        if game.end(state):
            return game.utility(state, player),infinity
        value, move = -infinity,infinity
        for possible_move in game.actions(state):
            possible_value,_ = min_value(game.make_move(state, possible_move))
            if possible_value > value:
                value, move = possible_value, possible_move
        return value, move

    def min_value(state):
        if game.end(state):
            return game.utility(state, player),-infinity
        value, move = +infinity,-infinity
        for possible_move in game.actions(state):
            possible_value,_ = max_value(game.make_move(state, possible_move))
            if possible_value < value:
                value, move = possible_value, possible_move
        return value, move
        

    return max_value(state)

In [15]:
play_game(TicTacToe(), dict(X=random_player, O=player(minimax_search))).utility

Player: X
Move: (1, 1)
. . .
. X .
. . .

Player: O
Move: (0, 0)
O . .
. X .
. . .

Player: X
Move: (2, 0)
O . X
. X .
. . .

Player: O
Move: (0, 2)
O . X
. X .
O . .

Player: X
Move: (0, 1)
O . X
X X .
O . .

Player: O
Move: (2, 1)
O . X
X X O
O . .

Player: X
Move: (2, 2)
O . X
X X O
O . X

Player: O
Move: (1, 0)
O O X
X X O
O . X

Player: X
Move: (1, 2)
O O X
X X O
O X X



0